In [10]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM

In [2]:
df = pd.DataFrame([
    ('u1', 'i1'), ('u1', 'i2'), ('u2', 'i1')
], columns=['uid', 'iid'])

df

,uid,iid
0,u1,i1
1,u1,i2
2,u2,i1


In [4]:
data = Dataset()
data.fit(
    users=df['uid'].unique().tolist(),
    items=df['iid'].unique().tolist()
)

interactions, weights = data.build_interactions(df[['uid', 'iid']].values.tolist())

In [5]:
rec = LightFM()
rec.fit(
    interactions=interactions,
    sample_weight=weights
)

In [50]:
np.argsort(
    rec.predict(
        user_ids=np.array([0, 0, 1, 1], dtype=np.int32),
        item_ids=np.array([0, 1, 0, 1], dtype=np.int32),
    ).reshape(-1, 2)
, axis=1)

array([[1, 0],
       [1, 0]])

In [37]:
(
    rec.predict(user_ids=0, item_ids=list(iid.values())),
    rec.predict(user_ids=1, item_ids=list(iid.values()))
)

(array([0.09381282, 0.07361589], dtype=float32),
 array([0.07186899, 0.05039217], dtype=float32))

In [32]:
uid, _, iid, _ = data.mapping()

In [33]:
iid

{'i1': 0, 'i2': 1}

In [44]:
iid_reverted = {v: k for k, v in iid.items()}
iid_reverted

{0: 'i1', 1: 'i2'}

In [35]:
iid.values()

dict_values([0, 1])

In [47]:
iid.keys()
[iid_reverted[k] for k in np.argsort(
    -rec.predict(user_ids=1, item_ids=list(iid.values()))
)[:2]]

['i1', 'i2']

In [51]:
rec.get_params()

{'loss': 'logistic',
 'learning_schedule': 'adagrad',
 'no_components': 10,
 'learning_rate': 0.05,
 'k': 5,
 'n': 10,
 'rho': 0.95,
 'epsilon': 1e-06,
 'max_sampled': 10,
 'item_alpha': 0.0,
 'user_alpha': 0.0,
 'random_state': RandomState(MT19937) at 0x7F9788254D40}

In [53]:
pd.Series([1, 2, 3]).tolist()

[1, 2, 3]

In [57]:
recs = []
N=2
for uid_ in ['u1', 'u2']:
    recs.append([
        iid_reverted[k]
        for k in np.argsort(-rec.predict(
            user_ids=uid.get(uid_),
            item_ids=list(iid.values())
        )[:N])
    ])

In [64]:
pd.Series(recs[0]).tolist()

['i1', 'i2']